In [43]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [44]:
!pip install wikipedia-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import pandas as pd
import numpy as np
import wikipediaapi
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [46]:
import pandas as pd
import wikipediaapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
data = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/articleDesc.csv", sep='[@$@]'  , engine='python', index_col = False)
print(data.columns)
print(data.head())

/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py:586: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return _read(filepath_or_buffer, kwds)


Index(['Article Name', 'Unnamed: 1', 'Unnamed: 2', 'Vital Article',
       'Unnamed: 4', 'Unnamed: 5', 'Level', 'Unnamed: 7', 'Unnamed: 8',
       'Class', 'Unnamed: 10', 'Unnamed: 11', 'Importance', 'Unnamed: 13',
       'Unnamed: 14', 'Topic', 'Unnamed: 16', 'Unnamed: 17', 'Wikiproject'],
      dtype='object')
                                    Article Name Unnamed: 1 Unnamed: 2  \
0  Population_history_of_ancient_Egypt_Archive_8        NaN        NaN   
1                                Sin_Mirar_Atrás        NaN        NaN   
2                                          RMWFC        NaN        NaN   
3                               Wilbur_E._Colyer        NaN        NaN   
4                                    Salmson_B.9        NaN        NaN   

  Vital Article Unnamed: 4  Unnamed: 5 Level  Unnamed: 7  Unnamed: 8  Class  \
0           NaN        NaN         NaN   NaN         NaN         NaN    NaN   
1           NaN        NaN         NaN   NaN         NaN         NaN    NaN   
2   

In [52]:
# Extract a sample of articles
np.random.seed(42)
sample_size = 500
df_sample = data.sample(sample_size)

# Initialize the Wikipedia API
wiki = wikipediaapi.Wikipedia('en')

In [53]:
def extract_features(article_name):
    page = wiki.page(article_name)
    if not page.exists():
        return None, None
    text = page.text
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha() and word.lower() not in stopwords.words('english')]
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    text = ' '.join(words)
    return text, page.summary

In [59]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
X = []
y = []
for article_name, article_class, *_ in df_sample.values:
    text, summary = extract_features(article_name)
    if text is not None and summary is not None:
        X.append(text + ' ' + summary)
        y.append(article_class)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [61]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [65]:
import pandas as pd
import wikipediaapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Load data
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/articleDesc.csv", sep='[@$@]'  , engine='python', index_col = False, header=None, names=['Article', 'Class'])
#df = pd.read_csv('dataset.csv', delimiter='@$@', header=None, names=['Article', 'Class'])
df = df.drop_duplicates()

# Sample articles
df_sample = df.sample(n=100, random_state=42)

# Extract article content using Wikipedia API
wiki = wikipediaapi.Wikipedia('en')
articles = []

for article_name in df_sample['Article']:
    page = wiki.page(article_name)
    if page.exists():
        articles.append(page.text)
    else:
        articles.append('')

# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
X = tfidf.fit_transform(articles)

# Create labels
y = df_sample['Class'].apply(lambda x: 1 if x == 'FA' else 0)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Test classifier
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)


/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py:586: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return _read(filepath_or_buffer, kwds)


Accuracy: 1.0
